In [ ]:
import os
import sys

app_path = os.path.abspath('..')
sys.path.insert(0, app_path)

import math
from datetime import datetime, UTC

from langchain_chroma import Chroma
from langchain import PromptTemplate
from langchain_ollama import ChatOllama
from pydantic.v1 import BaseModel, Field

from app.chroma_client import get_embeddings, get_client, Document

In [ ]:
channel_name = 'go_to_vilnius'
channel_id = -1133953167

In [ ]:
# ru_model_name = "cointegrated/rubert-tiny2"
ru_model_name = "intfloat/multilingual-e5-large"
embeddings = get_embeddings(ru_model_name)

chroma_client_from_telegram = get_client(f'telegram_{channel_name}', embeddings)

In [ ]:
llm_qwen3_8b = ChatOllama(model="qwen3:8b")
llm_llama_3b = ChatOllama(model="llama3.2:3b")

In [ ]:
# class UserQueryCategories(BaseModel):
#     category: str = Field(
#         description="The category of the query, the options are: Factual, Analytical, Opinion, or Contextual",
#         example="Factual"
#     )

class UserQueryCategories(BaseModel):
    category: str = Field(
        description="Категория запроса, возможные варианты: Фактический, Аналитический, Оценочный, Контекстуальный, Агрегированный и Обобщающий",
        example="Фактический"
    )

class UserQueryClassifier:
    def __init__(self, llm_model: ChatOllama = None):
        self.llm = llm_model or ChatOllama(model="llama3.2:3b")
        # self.prompt = PromptTemplate(
        #     input_variables=["query"],
        #     template="Classify the following query into one of these categories: Factual, Analytical, Opinion, or Contextual.\nQuery: {query}\nCategory:"
        # )
        self.prompt = PromptTemplate(
            input_variables=["query"],
            template="""Вы — умный классификатор пользовательских запросов.
Вам нужно отнести входящее сообщение ровно к одной из категорий:

- Фактический
- Аналитический
- Оценочный
- Контекстуальный
- Агрегированный
- Обобщающий

Примеры:

• «Когда последний день подачи налоговой декларации?» → Фактический
• «Как получить “детские деньги”?» → Аналитический
• «Какие самые позитивные сообщения в этом чате?» → Оценочный
• «Какие основные проблемы у мигрантов на Кипре?» → Контекстуальный
• «Сколько сообщений с хэштегом #вакансии опубликовано за март?» → Агрегированный
• «Какие основные топики обсуждаются в этом чате?» → Обобщающий

Теперь классифицируй следующее сообщение:
Запрос: «{query}»
Категория:"""
        )
        self.chain = self.prompt | self.llm.with_structured_output(UserQueryCategories)

    def classify(self, query):
        category = self.chain.invoke(query).category
        print(f"Query {query} classified as: {category}")
        return category

In [ ]:
class BaseRetrievalStrategy:
    def get_disclamer(self):
        return ''
    
    def retrieve(self, query) -> list[tuple[Document, float]]:
        return []


class SimpleRetrievalStrategy(BaseRetrievalStrategy):
    def __init__(self, db: Chroma):
        self.k = 10
        self.db = db

    def retrieve(self, query) -> list[tuple[Document, float]]:
        print("retrieving from DB only")
        docs_with_score = self.db.similarity_search_with_relevance_scores(query, k=self.k)
        return docs_with_score

In [ ]:
class SubQueries(BaseModel):
    sub_queries: list[str] = Field(
        description="List of sub-queries for comprehensive analysis",
        example=["What is the population of New York?", "What is the GDP of New York?"]
    )

class HypotheticalQuestionsRetrievalStrategy(BaseRetrievalStrategy):
    def __init__(self, db: Chroma, multi_shot_retrieval = True):
        self.k = 10
        self.db = db
        self.llm = llm_llama_3b
        self.multi_shot_retrieval = multi_shot_retrieval

    def similarity_search_for_each_question(self, queries) -> list[tuple[Document, float]]:
        docs_with_score = []
        k = math.ceil(self.k / len(queries))
        for query in queries:
            docs_with_score.extend(self.db.similarity_search_with_relevance_scores(query, k=k))
            print(f'Query {query} related docs: {'\n'.join(str(docs_with_score))}')

        return docs_with_score

    def similarity_search_all_at_once(self, queries) -> list[tuple[Document, float]]:
        docs_with_score = []
        query = ' '.join(queries)
        docs_with_score = self.db.similarity_search_with_relevance_scores(query, k=self.k)
        print(f'Query {query} related docs: {docs_with_score}')

        return docs_with_score
    
    def retrieve(self, query):
        print("retrieving analytical")
        # sub_queries_prompt = PromptTemplate(
        #     input_variables=["query", "k"],
        #     template="Generate {k} sub-questions for: {query}"
        # )
        sub_queries_prompt = PromptTemplate(
            input_variables=["query", "k"],
            template="Проанализируй следующий вопрос пользователя. \
Сгенерируй {k} альтернативных формулировок или связанных под-вопросов (sub_queries), которые рассматривают этот вопрос с разных сторон \
(например, уточняют детали, рассматривают причины, следствия, альтернативные сценарии и т.д.). \
Эти вопросы будут использованы для улучшения поиска в векторной базе знаний. \
Вопрос пользователя: '{query}'\n\n\
Альтернативные вопросы (sub_queries):",
        )

        sub_queries_chain = sub_queries_prompt | self.llm.with_structured_output(SubQueries)

        input_data = {"query": query, "k": self.k}
        sub_queries = sub_queries_chain.invoke(input_data).sub_queries
        print(f'sub queries for comprehensive analysis: {sub_queries}')

        queries = sub_queries
        queries.append(query)
        print(queries)
        if self.multi_shot_retrieval:
            docs_with_score = self.similarity_search_for_each_question(queries)
        else:
            docs_with_score = self.similarity_search_all_at_once(queries)

        return docs_with_score

    # def apply_diversity():
    #     # Use LLM to ensure diversity and relevance
    #     diversity_prompt = PromptTemplate(
    #         input_variables=["query", "docs", "k"],
    #         template="""Select the most diverse and relevant set of {k} documents for the query: '{query}'\nDocuments: {docs}\n
    #         Return only the indices of selected documents as a list of integers."""
    #     )
    #     diversity_chain = diversity_prompt | self.llm.with_structured_output(SelectedIndices)
    #     docs_text = "\n".join([f"{i}: {doc.page_content[:50]}..." for i, doc in enumerate(all_docs)])
    #     input_data = {"query": query, "docs": docs_text, "k": k}
    #     selected_indices_result = diversity_chain.invoke(input_data).indices
    #     print(f'selected diverse and relevant documents')

    #     return [all_docs[i] for i in selected_indices_result if i < len(all_docs)]

In [ ]:
class HypotheticalAnswers(BaseModel):
    answers: list[str] = Field(
        description="List of hypothetical answers for better information retrieval from vector database",
        # example=["What is the population of New York?", "What is the GDP of New York?"]
    )

class HypotheticalAnswersRetrievalStrategy(BaseRetrievalStrategy):
    def __init__(self, db: Chroma):
        self.k = 8
        self.db = db
        self.llm = llm_qwen3_8b  # llm_llama_3b

    def similarity_search_all_at_once(self, queries) -> list[tuple[Document, float]]:
        docs_with_score = []
        # if isinstance(queries, list):
            # query = '/n'.join(queries)
        query = queries
        docs_with_score = self.db.similarity_search_with_relevance_scores(query, k=self.k)
        print(f'Query {query} related docs: {docs_with_score}')

        return docs_with_score
    
    def retrieve(self, query):
        prompt_expanding = PromptTemplate(
            input_variables=["user_query", "k"],
            template="""
Контекст данных:
– Пользовательский запрос: {user_query}

Задача:
Сгенерировать ровно {k} гипотетических, но практически релевантных ответов (answers) на основе вышеуказанного вопроса.

Требования к каждому ответу:
1. Длина не более 2–3 предложений.
2. Формат выходных данных: маркированный.
3. Не добавлять заголовки, авторов, ссылки или другие метаданные.

Пример:
Вопрос: «Какие самые позитивные сообщения в чате?»
Ответ:
- Ого, какая замечательная идея! Это точно поднимет настроение всем!
- Спасибо за вашу помощь — вы просто спасли мой день!
- Мне очень нравится, как вы подходите к решению задач: вдохновляет!
- Ваше сообщение заставило меня улыбнуться — вы лучик света в этом чате!
- Благодарю за поддержку, с вами так легко общаться!

Гипотетические ответы (answers):""",
        )

        chain = prompt_expanding | self.llm.with_structured_output(HypotheticalAnswers)

        input_data = {"user_query": query, "k": self.k}
        additional_answers = chain.invoke(input_data).answers
        print(f'generated hypothetical answers: {additional_answers}')

        user_query_expanded = f'{query}\n\n{additional_answers}'
        print(user_query_expanded)
        docs_with_score = self.similarity_search_all_at_once(user_query_expanded)

        return docs_with_score

In [ ]:
class GraphRetrievalStrategy(SimpleRetrievalStrategy):
    def __init__(self):
        pass
    
    def get_disclamer(self):
        return '**The system has not implemented Query-Focused Summarization and Generalization yet.\n\
The follow answer is base only on simple retrival from DB and can be low quality.**\n'

In [ ]:
class AggregationRetrievalStrategy(BaseRetrievalStrategy):
    def __init__(self):
        pass
    
    def get_disclamer(self):
        return '**The system has not implemented Aggregation yet.**\n'

In [ ]:
user_query = "Какие самые токсичные сообщения в чате?"
# user_query = "Какие частые горячие темы в этом чате?"
# user_query = "Что известно о непродлении ВНЖ?"
# user_query = "Собери и обобщи всю информацию касаемо саун?"
# user_query = "Какие самые негативные сообщения с Литве и Вильнюсе в чате."
# user_query = "Можно ли компенсировать отель или аквапарк через велнес/БТА/бта/BTA?"
# user_query = "Как получить детские деньги или компенсацию по уходу за ребенком."
# user_query = "Какой процент сообщений в этом чате?"

In [ ]:
classifier = UserQueryClassifier(llm_llama_3b)
query_type = classifier.classify(user_query)

In [ ]:
strateg = HypotheticalAnswersRetrievalStrategy(db=chroma_client_from_telegram)

In [ ]:
strateg.retrieve(user_query)

In [ ]:
class AdaptiveRAG:
    def __init__(self, classifier, db):
        self.classifier = classifier
        self.strategies_to_query_type = {
            'Фактический': SimpleRetrievalStrategy(db=db),
            'Аналитический': HypotheticalQuestionsRetrievalStrategy(db=db, multi_shot_retrieval=False),
            'Оценочный': HypotheticalAnswersRetrievalStrategy(db=db),
            'Контекстуальный': HypotheticalQuestionsRetrievalStrategy(db=db),
            'Обобщающий': GraphRetrievalStrategy(),
            'Агрегированный': AggregationRetrievalStrategy(),
        }
        self.answer_template = "Query was classified as {query_type}.\n {disclamer}{answer}"
        self.llm = llm_qwen3_8b
        
    def get_answer(self, user_query: str) -> str:
        answer = ""
        query_type = self.classifier.classify(user_query)

        strategy = self.strategies_to_query_type.get(query_type)
        if not strategy:
            answer = "We can't classify your query. Try to refolmulate your question."
            return answer

        disclamer = strategy.get_disclamer()
        docs_with_score = strategy.retrieve(user_query)
        if not docs_with_score:
            return self.answer_template.format(query_type=query_type, disclamer=disclamer, answer='')
        
        filtered_related_docs = filter(lambda doc_score: doc_score[-1] > 0.3, docs_with_score)
        context = "\n\n---\n\n".join(
            f"{datetime.fromtimestamp(doc.metadata['date'], UTC)} - {doc.page_content}" for doc, _score in filtered_related_docs
        )

        final_prompt = PromptTemplate(
        input_variables=["context", "user_query"],
        template="""
Ты полезный AI ассистент, который отвечает на вопросы пользователя на основе контекста.
Контекст это релевантные вопросу сообщения из телеграм чата.

Контекст:
{context}

Вопрос пользователя:
{user_query}

Если в контексте недостаточно информации, чтобы ответить на вопрос пользователя, то скажи, что недостаточно информации.

Ответ:""",
        )

        chain = final_prompt | self.llm
        llm_response = chain.invoke({"user_query": user_query, "context": context}).content
        # return query_type, disclamer, llm_response
        llm_response = llm_response.split('</think>', 1)[-1]
        return self.answer_template.format(query_type=query_type, disclamer=disclamer, answer=llm_response)


In [ ]:
user_query = "Какие самые токсичные сообщения в чате?"
# user_query = "Какие частые горячие темы в этом чате?"
user_query = "Что известно о не продлении ВНЖ?"
# user_query = "Собери и обобщи всю информацию касаемо саун?"
# user_query = "Какие самые негативные сообщения с Литве и Вильнюсе в чате."
user_query = "Можно ли компенсировать отель или аквапарк через велнес/БТА/бта/BTA?"
user_query = "Как получить детские деньги или компенсацию по уходу за ребенком."
user_query = "Расскажи, что ты знаешь про получение детских денег или компенсации по уходу за ребенком."
user_query = "Сколько экстремистов в этом чате?"

In [ ]:
rag = AdaptiveRAG(classifier=classifier, db=chroma_client_from_telegram)
response = rag.get_answer(user_query)

In [ ]:
response